# mosaic_velocity
- Extract median velocity for single glacier from larger object

In [ ]:
#| default_exp mosaic_velocity

In [ ]:
#|hide
from nbdev.showdoc import *
import nbdev
from nbdev import nbdev_export

In [ ]:
#| export 
import os
import json

import geopandas as gpd
import numpy as np
import xarray as xr
import pandas as pd
import rioxarray as rxr

import geoviews as gv
import hvplot.pandas
import hvplot.xarray
import matplotlib.pyplot as plt

In [ ]:
from itslivetools.s3access import *

Read in RGI data:

In [ ]:
rgi_path = '/home/emmamarshall/Desktop/data/rgi/south_asia_west_14/14_rgi60_SouthAsiaWest.shp'
rgi = gpd.read_file(rgi_path)
rgi = rgi.to_crs('ESRI:102027')

Need to read in netcdf files foreach var of the clipped itslive var

In [ ]:
itslive_crops_mosaic_dir = '/home/emmamarshall/Desktop/data/itslive_mosaic/utm_zone43_crops/'

In [ ]:
#| export
def read_in_itslive_vars(path, crs):

    '''takes path to directory where cropped data var objects stored (output of gdalwarp fn)
    '''

    #make list of all files in dir
    ls = os.listdir(path)

    xr_ls = []
    #for each file in dir, read in and rename var 
    for var in range(len(ls)):
        ds = xr.open_dataset(os.path.join(path, ls[var]), chunks='auto')
        ds = ds.rename({'Band1':ls[var].split('_mosaic')[0]})
        xr_ls.append(ds)
        
    ds_allvars = xr.merge(xr_ls)

    ds_allvars.rio.write_crs(crs, inplace=True)
    return ds_allvars

In [ ]:
ds = read_in_itslive_vars(itslive_crops_mosaic_dir, 'ESRI:102027')

In [ ]:
ds_bbox = get_bbox(ds, 'ESRI:102027')

Subset RGI to bounds of itslive object:

In [ ]:
rgi = gpd.sjoin(rgi, ds_bbox, how='inner')

# Clip to ind glacier

In [ ]:
#| export
def extract_glacier_velocity_ds(rgi_gpdf, itslive_ds, rgiid):

    rgi_single = rgi_gpdf.loc[rgi_gpdf['RGIId'] == rgiid]

    ds = itslive_ds.rio.clip(rgi_single.geometry, rgi_single.crs)

    return ds

In [ ]:
ds = extract_glacier_velocity_ds(rgi, ds, 'RGI60-14.24214')

In [ ]:
#| export
def calc_median_velocity(ds):
    med = ds.v.load().median()
    return med #m/y